# 2024 NFL Statistics 

## 1. Introduction
- Get current statistics for the 2024 season on inividual player rankings and team statistics

## 2. Data Loading
### 2.1: Load database and notebook settings
- Load necessary modules and nfl_fantasy.db (sqlite) into notebook

## 3. Player Statistics
### 3.1: Rushing Statistics
### 3.2: Receiving Statistics
### 3.3: Passing Statistics
### 3.4: Positional Fantasy Rankings

## 4. Team Statistics
### 4.1: Offensive Team Statistics
### 4.2: Defensive Team Statistics



In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import seaborn as sns
import os

## 2. Data Loading
### 2.1: Load database and notebook settings
- Load necessary modules and nfl_fantasy.db (sqlite) into notebook


In [2]:
%load_ext sql
conn = sqlite3.connect('..\\..\\nfl_fantasy.db')
cursor = conn.cursor()
%sql sqlite:///C:\Users\Jrdes\Desktop\NFL\NFL-Analytics\nfl_fantasy.db

pd.set_option('display.max_rows', 100) # Increase pandas to display 100 rows max
pd.set_option('display.max_columns', 25)

## 3. Player Statistics
### 3.1: Rushing Statistics

In [11]:
%%sql db_player_rush_stats <<

SELECT 
    pgs.PLAYER_NAME AS Player,
    p.POSITION AS Position,
    pgs.TEAM AS Team,
    SUM(pgs.RUSHING_CARRIES) AS Carries,
    SUM(pgs.RUSHING_RUSH_YARDS) AS Rushing_Yards,
    SUM(pgs.RUSHING_RUSH_TOUCHDOWNS) AS Rushing_TDs,
    ROUND(SUM(pgs.RUSHING_RUSH_YARDS * 1.0) / SUM(pgs.RUSHING_CARRIES), 2) AS YPC
FROM 
    Player_Game_Stats pgs
    INNER JOIN Game g on g.GAME_ID = pgs.GAME_ID
    INNER JOIN Player p on p.PLAYER_ID = pgs.PLAYER_ID
WHERE
    g.GAME_TYPE = "Regular Season"
    AND
    g.SEASON_ID = "2024"
    AND
    pgs.RUSHING_CARRIES > 0
GROUP BY
    pgs.PLAYER_NAME,
    pgs.TEAM

ORDER BY 
    RUSHING_YARDS DESC;

 * sqlite:///C:\Users\Jrdes\Desktop\NFL\NFL-Analytics\nfl_fantasy.db
Done.
Returning data to local variable db_player_rush_stats


### 3.2: Receiving Statistics

In [10]:
%%sql db_player_rec_stats <<

SELECT 
    pgs.PLAYER_NAME AS Player,
    p.POSITION AS Position,
    pgs.TEAM AS Team,
    SUM(pgs.RECEIVING_TARGETS) AS Targets,
    SUM(pgs.RECEIVING_RECEPTIONS) AS Receptions,
    SUM(pgs.RECEIVING_REC_YARDS) AS Receiving_Yards,
    SUM(pgs.RECEIVING_REC_TOUCHDOWNS) AS Receiving_TDs,
    ROUND(SUM(pgs.RECEIVING_REC_YARDS * 1.0) / SUM(pgs.RECEIVING_RECEPTIONS), 2) AS YPC
FROM 
    Player_Game_Stats pgs
    INNER JOIN Game g on g.GAME_ID = pgs.GAME_ID
    INNER JOIN Player p on p.PLAYER_ID = pgs.PLAYER_ID
WHERE
    g.GAME_TYPE = "Regular Season"
    AND
    g.SEASON_ID = "2024"
    AND
    pgs.RECEIVING_RECEPTIONS > 0
GROUP BY
    pgs.PLAYER_NAME,
    pgs.TEAM

ORDER BY 
    Receiving_Yards DESC;

 * sqlite:///C:\Users\Jrdes\Desktop\NFL\NFL-Analytics\nfl_fantasy.db
Done.
Returning data to local variable db_player_rec_stats


### 3.3: Passing Statistics

In [24]:
%%sql db_player_pass_stats <<

SELECT 
    pgs.PLAYER_NAME AS Player,
    p.POSITION AS Position,
    pgs.TEAM AS Team,
    SUM(pgs.PASSING_ATTEMPTS) AS Attempts,
    SUM(pgs.PASSING_COMPLETIONS) AS Completions,
    SUM(pgs.PASSING_YARDS) AS Passing_Yards,
    SUM(pgs.PASSING_TOUCHDOWNS) AS Passing_TDs,
    SUM(pgs.PASSING_INTERCEPTIONS) AS Interceptions,
    ROUND(SUM(pgs.PASSING_COMPLETIONS * 1.0) / SUM(pgs.PASSING_ATTEMPTS), 2) AS Completion_Percentage
FROM 
    Player_Game_Stats pgs
    INNER JOIN Game g on g.GAME_ID = pgs.GAME_ID
    INNER JOIN Player p on p.PLAYER_ID = pgs.PLAYER_ID
WHERE
    g.GAME_TYPE = "Regular Season"
    AND
    g.SEASON_ID = "2024"
    AND
    pgs.PASSING_ATTEMPTS > 0
GROUP BY
    pgs.PLAYER_NAME,
    pgs.TEAM

ORDER BY 
    Passing_TDs DESC;

 * sqlite:///C:\Users\Jrdes\Desktop\NFL\NFL-Analytics\nfl_fantasy.db
Done.
Returning data to local variable db_player_pass_stats


In [26]:

player_rush_stats_df = db_player_rush_stats.DataFrame()
player_rec_stats_df = db_player_rec_stats.DataFrame()
player_pass_stats_df = db_player_pass_stats.DataFrame()
player_pass_stats_df

,Player,Position,Team,Attempts,Completions,Passing_Yards,Passing_TDs,Interceptions,Completion_Percentage
0,Baker Mayfield,QB,TB,234,165,1859,18,7,0.71
1,Jordan Love,QB,GB,179,110,1351,15,8,0.61
2,Lamar Jackson,QB,BAL,198,135,1810,15,2,0.68
3,Joe Burrow,QB,CIN,226,159,1759,14,2,0.70
4,Josh Allen,QB,BUF,189,119,1483,12,0,0.63
5,Sam Darnold,QB,MIN,164,109,1370,12,5,0.66
6,Aaron Rodgers,QB,NYJ,256,158,1663,10,7,0.62
7,C.J. Stroud,QB,HOU,229,152,1663,10,4,0.66
8,Jared Goff,QB,DET,174,128,1610,10,4,0.74
9,Kirk Cousins,QB,ATL,242,162,1830,10,7,0.67
